In [1]:
import cv2
import glob
import numpy as np
from skimage.transform import resize

img_length = 60

def process_img(img_name):
    img = cv2.imread(img_name)
    
    mask = np.zeros(img.shape[:2], np.uint8)
    rect = (0, 0, img_length, img_length)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 6, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    img = img * mask2[:, :, np.newaxis]
    
    resized_img = resize(img, (img_length, img_length, 3), preserve_range=True)
    return np.array(resized_img).astype(int)

full_thickness_img_names = glob.glob('./images/full_thickness/*.jpg') + glob.glob('./images/full_thickness/*.JPG')
X1 = [process_img(img_name) for img_name in full_thickness_img_names]
y1 = [0] * len(X1)

partial_thickness_img_names = glob.glob('./images/partial_thickness/*.jpg') + glob.glob('./images/partial_thickness/*.JPG')
X2 = [process_img(img_name) for img_name in partial_thickness_img_names]
y2 = [1] * len(X2)

superficial_dermal_img_names = glob.glob('./images/superficial_dermal/*.jpg') + glob.glob('./images/superficial_dermal/*.JPG')
X3 = [process_img(img_name) for img_name in superficial_dermal_img_names]
y3 = [2] * len(X3)

X = np.concatenate((X1, X2, X3))

raw_y = np.concatenate((y1, y2, y3))

In [2]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

X_train, X_test, raw_y_train, raw_y_test = train_test_split(X, raw_y, test_size=0.3)
y_train = to_categorical(raw_y_train)
y_test = to_categorical(raw_y_test)

Using TensorFlow backend.


In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

img_width, img_height = img_length, img_length

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=25, epochs=125, verbose=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/125
44/44 [==============================] - 1s 15ms/step - loss: 4.3844 - accuracy: 0.3409
Epoch 2/125
44/44 [==============================] - 0s 5ms/step - loss: 4.0810 - accuracy: 0.4773
Epoch 3/125
44/44 [==============================] - 0s 5ms/step - loss: 4.1426 - accuracy: 0.4091
Epoch 4/125
44/44 [==============================] - 0s 8ms/step - loss: 3.1871 - accuracy: 0.3864
Epoch 5/125
44/44 [==============================] - 0s 10ms/step - loss: 3.5475 - accuracy: 0.4318
Epoch 6/125
44/44 [==============================] - 0s 6ms/step - loss: 3.3065 - accuracy: 0.5000
Epoch 7/125
44/44 [==============================] - 0s 7ms/step - loss: 3.0293 - accuracy: 0.5227
Epoch 8/125
44/44 [==============================] - 0s 8ms/step - loss: 1.5484 - accuracy: 0.6364
Epoch 9/125
44/44 [==============================] - 0s 6ms/step - loss: 1.5167 - accu

44/44 [==============================] - 0s 4ms/step - loss: 0.2700 - accuracy: 0.8864
Epoch 78/125
44/44 [==============================] - 0s 4ms/step - loss: 0.2687 - accuracy: 0.8409
Epoch 79/125
44/44 [==============================] - 0s 5ms/step - loss: 0.2768 - accuracy: 0.8864
Epoch 80/125
44/44 [==============================] - 0s 8ms/step - loss: 0.3240 - accuracy: 0.8182
Epoch 81/125
44/44 [==============================] - 0s 5ms/step - loss: 0.2954 - accuracy: 0.8636
Epoch 82/125
44/44 [==============================] - 0s 9ms/step - loss: 0.2912 - accuracy: 0.8409
Epoch 83/125
44/44 [==============================] - 0s 9ms/step - loss: 0.2858 - accuracy: 0.8409
Epoch 84/125
44/44 [==============================] - 0s 5ms/step - loss: 0.2898 - accuracy: 0.7955
Epoch 85/125
44/44 [==============================] - 0s 3ms/step - loss: 0.2664 - accuracy: 0.8636
Epoch 86/125
44/44 [==============================] - 0s 3ms/step - loss: 0.2649 - accuracy: 0.8864
Epoch 87/125


In [4]:
from sklearn.metrics import classification_report

y_pred_train = model.predict(X_train)
y_pred_train_bool = np.argmax(y_pred_train, axis=1)
print(classification_report(raw_y_train, y_pred_train_bool, target_names=['Full Thickness', 'Partial Thickness', 'Superficial Dermal']))
      
y_pred_test = model.predict(X_test)
y_pred_test_bool = np.argmax(y_pred_test, axis=1)
print(classification_report(raw_y_test, y_pred_test_bool, target_names=['Full Thickness', 'Partial Thickness', 'Superficial Dermal']))

                    precision    recall  f1-score   support

    Full Thickness       1.00      0.73      0.85        15
 Partial Thickness       0.73      1.00      0.84        16
Superficial Dermal       1.00      0.85      0.92        13

          accuracy                           0.86        44
         macro avg       0.91      0.86      0.87        44
      weighted avg       0.90      0.86      0.87        44

                    precision    recall  f1-score   support

    Full Thickness       0.62      0.62      0.62         8
 Partial Thickness       0.57      0.57      0.57         7
Superficial Dermal       1.00      1.00      1.00         4

          accuracy                           0.68        19
         macro avg       0.73      0.73      0.73        19
      weighted avg       0.68      0.68      0.68        19

